In [1]:
import os, json, sys, shutil
from ovejero import model_trainer

2023-09-16 23:05:24.465732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 23:05:25.140365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/parlange/anaconda3/envs/bnn/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


# Fitting a Model Using model_trainer

__Author:__ Sebastian Wagner-Carena

__Last Run:__ 08/04/2020

__Goals:__ Learn how to use model_trainer to fit the types of models used by ovejero

__Before running this notebook:__ Run the Generate_Config notebook to understand what goes into the configuration files for overjero

We'll start by loading up the test configuration file made by Generate_Config and inspecting it.

In [3]:
json_path = '/home/parlange/ovejero/test/test_data/' + 'test.json'
with open(json_path,'r') as json_f:
    cfg = json.load(json_f)
print(cfg)

{'training_params': {'bnn_type': 'gmm', 'dropout_type': 'standard', 'batch_size': 10, 'n_epochs': 10, 'learning_rate': 0.0001, 'decay': 3e-06, 'kernel_regularizer': 1e-05, 'dropout_rate': 0.1, 'dropout_regularizer': 1e-06, 'root_path': '/home/parlange/ovejero/test/test_data/', 'tf_record_path': 'tf_record_test', 'final_params': ['external_shear_g1', 'external_shear_g2', 'lens_mass_center_x', 'lens_mass_center_y', 'lens_mass_e1', 'lens_mass_e2', 'lens_mass_gamma', 'lens_mass_theta_E_log'], 'flip_pairs': [], 'img_dim': 128, 'model_weights': '/home/parlange/ovejero/test/test_data/test_model.h5', 'tensorboard_log_dir': '/home/parlange/ovejero/test/test_data/test.log', 'random_seed': 1138, 'norm_images': True, 'shift_pixels': 2, 'shift_params': [['lens_mass_center_x'], ['lens_mass_center_y']], 'pixel_scale': 0.051, 'baobab_config_path': '/home/parlange/ovejero/test/test_data/test_baobab_cfg.py'}, 'validation_params': {'root_path': '/home/parlange/ovejero/test/test_data/', 'tf_record_path': 

A lot of good information there! This is a good config file to start with. Let's go ahead and change a few paths and use it for our toy model.

In [10]:
# Change the model weights to point to the demo directory! Same for log file.
print('old path:')
print(cfg['training_params']['model_weights'])

cfg['training_params']['model_weights'] = '/home/parlange/ovejero/test/test_data/test_model.h5'
cfg['training_params']['tensorboard_log_dir'] = '/home/parlange/ovejero/test/test_data/test_logs'
cfg['training_params']['baobab_config_path'] = '/home/parlange/ovejero/test/test_data/test_baobab_cfg.py'
cfg['training_params']['root_path'] = '/home/parlange/ovejero/test/test_data/'
cfg['validation_params']['root_path'] = '/home/parlange/ovejero/test/test_data/'
print('new path:')
print(cfg['training_params']['model_weights'])

# Don't want shifts for this easier version of the problem
cfg['training_params']['shift_pixels'] = 0

# Also let's start with the easy diagonal case
cfg['training_params']['bnn_type'] = 'diag'

# Now let's go ahead and save this as our new configuration file
diag_json_path = '/home/parlange/ovejero/test/test_data/diag.json'
with open(diag_json_path,'w') as json_f:
    json.dump(cfg,json_f,indent=4)

old path:
/home/parlange/ovejero/test/test_data/test_model.h5
new path:
/home/parlange/ovejero/test/test_data/test_model.h5


All we have to do is call the main function of model_trainer with the path to our config file! You should see the loss go down as the model learns to overfit to the lenses in the very small training set. Because the random seed is set by the configuration file the final loss should be 1.9520

In [11]:
# This is equivalent to 'python -m model_trainer diag_json_path' in the terminal where diag_json_path is the path.
sys.argv = ['model_trainer',diag_json_path]
model_trainer.main()

Checking for training data.
TFRecord found at /home/parlange/ovejero/test/test_data/tf_record_test
Checking for validation data.
TFRecord found at /home/parlange/ovejero/test/test_data/tf_record_test_val
Initializing the model
Is model built: True
Loaded weights /home/parlange/ovejero/test/test_data/test_model.h5
Epoch 1/10
2/2 [==============================] - ETA: 0s - loss: 4.8372 - diagonal_covariance_loss: 4.7316 - mse_loss: 0.9990

/home/parlange/anaconda3/envs/bnn/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 5s 3s/step - loss: 4.8372 - diagonal_covariance_loss: 4.7316 - mse_loss: 0.9990 - val_loss: 4.7427 - val_diagonal_covariance_loss: 4.6381 - val_mse_loss: 1.1559
Epoch 2/10
2/2 [==============================] - 3s 2s/step - loss: 4.7325 - diagonal_covariance_loss: 4.6283 - mse_loss: 1.1665 - val_loss: 4.0333 - val_diagonal_covariance_loss: 3.9304 - val_mse_loss: 0.9889
Epoch 3/10
2/2 [==============================] - 3s 2s/step - loss: 3.7269 - diagonal_covariance_loss: 3.6244 - mse_loss: 0.9158 - val_loss: 3.6855 - val_diagonal_covariance_loss: 3.5844 - val_mse_loss: 0.9311
Epoch 4/10
2/2 [==============================] - 3s 2s/step - loss: 3.6939 - diagonal_covariance_loss: 3.5934 - mse_loss: 0.8874 - val_loss: 3.6253 - val_diagonal_covariance_loss: 3.5262 - val_mse_loss: 0.9280
Epoch 5/10
2/2 [==============================] - 3s 2s/step - loss: 3.5517 - diagonal_covariance_loss: 3.4532 - mse_loss: 0.9441 - val_loss: 2.7749 - val_diagonal_cov

At the end of each epoch we get the loss (which includes the concrete dropout regularization penalty) and the diagonal/full/gmm loss term (essentially the measure of how well our pdf is doing at capturing the data) on both the training and validation sets. 

In [ ]:
# Clean up the files that were created by this notebook.
test_data_path = os.getcwd()[:-5]+'test/test_data/'

os.remove(test_data_path+'new_metadata.csv')
os.remove(test_data_path+'norms.csv')
os.remove(test_data_path+'tf_record_test')
os.remove(test_data_path+'tf_record_test_val')
os.remove('test_model.h5')
shutil.rmtree('test_logs')
os.remove('diag.json')

If we want to fit a different type of model, all we have to do is change the config specification for the bnn type or dropout rate.